In [ ]:
### Header
# Jared Gibbs
# 3/21/25
# Property Tax data

In [ ]:
### Libraries and Imports

import pandas as pd
import warnings

warnings.filterwarnings("ignore", category = FutureWarning)

df_property_rawdata = pd.read_excel("document (79).xls")


In [ ]:
### Data Cleaning/ pre-processing

# statewide tax rates

# List of desired column names
column_names = ['Tax Year', 'county_id', 'entity_id', 'Real Tax Rate', 'Personal Tax Rate',
                'total_real_land', 'total_real_building', 'Personal Property', 'Total Property', 'Total Real Property', 
                'Total_Personal', 'Centrally Assessed', 'Total SCME', 'prime_resid_land', 'nonprime_reside_land', 
                'commercial_land', 'FAA_land', 'Unimproved', 'prime_resid_building', 'nonprime_reside_building', 
                'commercial_building', 'agriculture_building', 'primary_mobile_home', 'secondary_mobile_home', 'SCME', 
                'Business Personal Property', 'mv_fee_in_lieu', 'mv_age_based', 'year_end_CA-gentax', 'total_real_land, count', 
                'total_real_building, count', 'Personal Property, count', 'Total Property, count', 'prime_resid_land, count', 'nonprime_reside_land, count', 
                'commercial_land, count', 'FAA_land, count', 'Unimproved, count', 'prime_resid_building, count', 'nonprime_reside_building, count', 
                'commercial_building, count', 'agriculture_building, count', 'primary_mobile_home, count', 'secondary_mobile_home, count','SCME, count',
                'Business Personal Property, count', 'mv_fee_in_lieu, count', 'mv_age_based, count', 'year_end_CA-gentax, count' 
                ]

# List of property types
property_types = ['Primary Residential', 'Non-primary Residential', 'Commercial', 'Agricultural', 'Unimproved', 'Total Real Property', 
                  'Personal Property', 
                  'Centrally Assessed', 
                  'Total Property']

property_types_building = ['prime_resid_building', 'nonprime_reside_building', 'commercial_building', 'agriculture_building', ]
property_types_land = ['prime_resid_land', 'nonprime_reside_land', 'commercial_land', 'FAA_land' ]
property_types_other = ['Unimproved', 'Personal Property']

property_counts_building = ['prime_resid_building, count', 'nonprime_reside_building, count', 'commercial_building, count', 'agriculture_building, count', ]
property_counts_land = ['prime_resid_land, count', 'nonprime_reside_land, count', 'commercial_land, count', 'FAA_land, count' ]
property_counts_other = ['Unimproved, count', 'Personal Property, count']

property_types_ext = property_types_building + property_types_land + property_types_other

property_counts_ext = property_counts_building + property_counts_land + property_counts_other


# Rename columns using desired names
df_cleandata = df_property_rawdata.set_axis(column_names, axis=1)

entities = df_cleandata['entity_id'].unique().tolist()

# split county/ entity name and code out from county/ entity id
df_cleandata['entity_code']=df_cleandata['entity_id'].str[0:4]
df_cleandata['entity_code']=df_cleandata['entity_code'].astype(int)

df_cleandata['county_code']=df_cleandata['county_id'].str[0:2]
df_cleandata['county_code']=df_cleandata['county_code'].astype(int)

# Create Statewide Variables

years = df_cleandata["Tax Year"].unique().tolist()

variables = df_cleandata.columns.to_list()
variables = variables[5:]


for year in years:
    df_cleandata.loc[-1]= [year, "99-STATEWIDE", "9999-STATEWIDE", 0, 0, 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                        0, 0, 0, 0, 0, 0]
    
    for variable in variables:

        alpha = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['entity_code'] == 1010 )][variable].sum()

        df_cleandata.loc[[-1],[variable]] = alpha

    df_cleandata.index = df_cleandata.index + 1


# split county/ entity name and code out from county/ entity id
df_cleandata['entity_code']=df_cleandata['entity_id'].str[0:4]
df_cleandata['entity_code']=df_cleandata['entity_code'].astype(int)

df_cleandata['county_code']=df_cleandata['county_id'].str[0:2]
df_cleandata['county_code']=df_cleandata['county_code'].astype(int)

# identify entitiy type by entity code 
df_cleandata.loc[df_cleandata['entity_code'] == 1010, 'entity_type'] = 'County'
df_cleandata.loc[df_cleandata['entity_code'] == 1015, 'entity_type'] = 'Multicounty Assessing'
df_cleandata.loc[df_cleandata['entity_code'] == 1020, 'entity_type'] = 'County Assessing'
df_cleandata.loc[(df_cleandata['entity_code'] >= 2000) & (df_cleandata['entity_code'] < 3000), 'entity_type'] = 'School District'
df_cleandata.loc[(df_cleandata['entity_code'] >= 3000) & (df_cleandata['entity_code'] < 4000), 'entity_type'] = 'Municipality'
df_cleandata.loc[(df_cleandata['entity_code'] >= 4000) & (df_cleandata['entity_code'] < 6000), 'entity_type'] = 'Special Service District'

df_cleandata.loc[(df_cleandata['entity_code'] >= 6000) & (df_cleandata['entity_code'] < 7000), 'entity_type'] = 'Special District'

df_cleandata.loc[(df_cleandata['entity_code'] >= 8000) & (df_cleandata['entity_code'] < 9999), 'entity_type'] = 'RDA or CDA'
df_cleandata.loc[df_cleandata['entity_code'] >= 9999, 'entity_type'] = 'Statewide'

df_cleandata['county_name']=df_cleandata['county_id'].str[3:].str.capitalize()

df_cleandata.loc[df_cleandata['entity_code'] == 1010, 'entity_name'] = df_cleandata['county_name']
df_cleandata.loc[df_cleandata['entity_code'] == 1015, 'entity_name'] = 'Multicounty Assessing' + ", " + df_cleandata['county_name']
df_cleandata.loc[df_cleandata['entity_code'] == 1020, 'entity_name'] = 'County Assessing' + ", " + df_cleandata['county_name']
df_cleandata.loc[(df_cleandata['entity_code'] >= 2000) & (df_cleandata['entity_code'] < 7000), 'entity_name'] = df_cleandata['entity_id'].str[5:].str.capitalize() + ", " + df_cleandata['county_name']
df_cleandata.loc[df_cleandata['entity_code'] == 9999, 'entity_name'] = 'Statewide'


# Combine land and buidlings by catagory
scale_factor = 1000000000
df_cleandata['Primary Residential'] = (df_cleandata['prime_resid_building'] + df_cleandata['prime_resid_land'])/scale_factor
df_cleandata['Non-primary Residential'] = (df_cleandata['nonprime_reside_building'] + df_cleandata['nonprime_reside_land'])/scale_factor
df_cleandata['Commercial'] = (df_cleandata['commercial_building'] + df_cleandata['commercial_land'])/scale_factor
df_cleandata['Agricultural'] = (df_cleandata['FAA_land'] + df_cleandata['agriculture_building'])/scale_factor
df_cleandata['Unimproved'] = df_cleandata['Unimproved']/scale_factor
df_cleandata['Personal Property'] = df_cleandata["Personal Property"]/scale_factor
df_cleandata['Centrally Assessed'] = df_cleandata["Centrally Assessed"]/scale_factor
df_cleandata['Total Property'] = (df_cleandata["Total Property"]/scale_factor) + df_cleandata['Centrally Assessed']

# Create average value variables
df_cleandata['prime_resid_building, average'] = df_cleandata['prime_resid_building'] / df_cleandata['prime_resid_building, count']
df_cleandata['prime_resid_land, average'] = df_cleandata['prime_resid_land'] / df_cleandata['prime_resid_land, count']

df_cleandata['Primary Residential, average'] = df_cleandata['prime_resid_building, average'] + df_cleandata['prime_resid_land, average']

df_cleandata['nonprime_reside_building, average'] = df_cleandata['nonprime_reside_building'] / df_cleandata['nonprime_reside_building, count']
df_cleandata['nonprime_reside_land, average'] = df_cleandata['nonprime_reside_land'] / df_cleandata['nonprime_reside_land, count']

df_cleandata['Non-primary Residential, average'] = df_cleandata['nonprime_reside_building, average'] + df_cleandata['nonprime_reside_land, average']

df_cleandata['commercial_building, average'] = df_cleandata['commercial_building'] / df_cleandata['commercial_building, count']
df_cleandata['commercial_land, average'] = df_cleandata['commercial_land'] / df_cleandata['commercial_land, count']

df_cleandata['Commercial, average'] = df_cleandata['commercial_building, average'] + df_cleandata['commercial_land, average']

df_cleandata['agricultural_building, average'] = df_cleandata['agriculture_building'] / df_cleandata['agriculture_building, count']
df_cleandata['agricultural_land, average'] = df_cleandata['FAA_land'] / df_cleandata['FAA_land, count']

df_cleandata['Agricultural, average'] = df_cleandata['agricultural_building, average'] + df_cleandata['agricultural_land, average']

df_cleandata['Unimproved, average'] = (df_cleandata['Unimproved'] * scale_factor) / df_cleandata['Unimproved, count']

df_cleandata['Personal Property, average'] = (df_cleandata['Personal Property'] * scale_factor) / df_cleandata['Personal Property, count']

#df_cleandata['Centrally Assessed, average'] = (df_cleandata['Centrally Assessed'] * scale_factor) / df_cleandata['Centrally Assessed, count']


# Create Share Variables
for property_type in property_types:
    df_cleandata[property_type + ", Percent"] = df_cleandata[property_type]/ df_cleandata['Total Property']


# Create Revenue Variables
revenue_variables = []

for property_type in property_types:
    if property_type == 'Personal Property':
        df_cleandata['Revenue, ' + property_type] = df_cleandata[property_type] * 1000 * df_cleandata['Personal Tax Rate']
   
    elif property_type != 'Personal Property':
        df_cleandata['Revenue, ' + property_type] = df_cleandata[property_type] * 1000 * df_cleandata['Real Tax Rate']

    revenue_variables+= ['Revenue, ' + property_type]


# Calcualte statewide revenue
for revenue_type in revenue_variables:

    for year in years:

        beta = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["entity_code"]!= 9999)][revenue_type].sum()
    
        df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["entity_code"]== 9999), revenue_type] = beta


# Calculate Statewide tax rate faced by taxpayers

for year in years:

    gamma = df_cleandata.loc[(df_cleandata["Tax Year"]== year) & (df_cleandata["entity_code"]== 9999)]["Revenue, Total Property"].sum()
    gamma = gamma/ (df_cleandata.loc[(df_cleandata["Tax Year"]== year) & (df_cleandata["entity_code"]== 9999)]["Total Property"].sum() * 1000 )
    
    df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["entity_code"]== 9999), ["Real Tax Rate"]] = gamma

    df_cleandata.loc[(df_cleandata["Tax Year"] == year + 1) & (df_cleandata["entity_code"]== 9999), ["Personal Tax Rate"]] = gamma


# Calculate Statewide tax rate charged by entities

df_cleandata["Real Tax Rate, entity"] = df_cleandata["Real Tax Rate"]
df_cleandata["Personal Tax Rate, entity"] = df_cleandata["Personal Tax Rate"]

for year in years:

    delta = df_cleandata.loc[(df_cleandata["Tax Year"]== year) & (df_cleandata["entity_code"]== 9999)]["Revenue, Total Property"].sum()
    delta = delta / (df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["entity_code"]!= 9999)]["Total Property"].sum() * 1000)

    df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["entity_code"]== 9999), ["Real Tax Rate, entity"]] = delta

    df_cleandata.loc[(df_cleandata["Tax Year"] == year + 1) & (df_cleandata["entity_code"]== 9999), ["Personal Tax Rate, entity"]] = delta


# Convert Tax Rates to Percents

df_cleandata["Real Tax Rate"] = df_cleandata["Real Tax Rate"] * 100
df_cleandata["Personal Tax Rate"] = df_cleandata["Personal Tax Rate"] * 100




In [ ]:
### Create YoY Growth Variables

df_cleandata['shifted'] = 0


for property_type in property_types_ext:
    for entity in entities:
        for year in years:

            epsilon = df_cleandata.loc[(df_cleandata['Tax Year']== year - 1 ) & (df_cleandata["entity_id"] == entity), [property_type]].sum()

            df_cleandata.loc[(df_cleandata['Tax Year']== year) & (df_cleandata["entity_id"] == entity), ['shifted']] = epsilon[0]

    df_cleandata.loc[df_cleandata["Tax Year"] == 2014, ['Total Growth, ' + property_type]] = 0
    
    df_cleandata.loc[df_cleandata["Tax Year"] != 2014, ['Total Growth, ' + property_type]] = (df_cleandata[property_type] / df_cleandata['shifted']) - 1

for property_type in property_counts_ext:
    for entity in entities:
        for year in years:

            epsilon = df_cleandata.loc[(df_cleandata['Tax Year']== year - 1 ) & (df_cleandata["entity_id"] == entity), [property_type]].sum()

            df_cleandata.loc[(df_cleandata['Tax Year']== year) & (df_cleandata["entity_id"] == entity), ['shifted']] = epsilon[0]

    df_cleandata.loc[df_cleandata["Tax Year"] == 2014, ['Count Growth, ' + property_type]] = 0
    
    df_cleandata.loc[df_cleandata["Tax Year"] != 2014, ['Count Growth, ' + property_type]] = (df_cleandata[property_type] / df_cleandata['shifted']) - 1




In [ ]:
for property_type in property_types_ext:
    df_cleandata['Value Growth, ' + property_type] = df_cleandata['Total Growth, ' + property_type] - df_cleandata['Count Growth, ' + property_type + ', count']

In [ ]:
### Export processed data

df_cleandata.to_csv("property_cleandata.csv")